# Download MEaSUREs Annual Antarctic Ice Velocity Maps, Version 1 (NSIDC-0720)


NSICD MEaSUREs Annual Antarctic Ice Velocity Maps, Version 1 (NSIDC-0720):

https://nsidc.org/data/nsidc-0720/versions/1  <br><br>



NASA EarthData Data Access (echo_collection_id=C2245171699-NSIDC_ECS):

https://search.earthdata.nasa.gov/search/granules/collection-details?p=C2245171699-NSIDC_ECS&pg[0][v]=f&pg[0][gsk]=-start_date&q=NSIDC-0720%20V001 


In [85]:
import numpy as np
import itertools
import requests
import os
#import netCDF4 as nc4
#from osgeo import gdal

## Set options

First, set collections ID for desired data set from NASA EarthData and set the region name.

Example: Getting a collection ID from EarthData Search: 
[MEaSUREs Annual Antarctic Ice Velocity Maps V001](https://search.earthdata.nasa.gov/search/granules/collection-details?p=C2245171699-NSIDC_ECS&pg[0][v]=f&pg[0][gsk]=-start_date&q=NSIDC-0720%20V001&tl=1686700071.247!3!!)

While viewing the collection on EarthData, as in the above link, follow "View More Info" to visit the CMR page for the collection.
Then, look for the collections ID in the URL or as a tag below the title. 

![Locating Collection ID from CMR page](getting_collectionID.png)


In [86]:
collection_id = 'C2245171699-NSIDC_ECS' # MEaSUREs Annual Antarctic
region_name = 'Antarctic'

Next, define two directories on your local drive as follows:

| directory | purpose | 
|-----------|---------|
|`project_folder` | This is the path where output data from the changes module will be stored - the data to be used directly for analysis. | 
|`data_folder` | This is the path where ice velocity and elevation data, from their respective sources, will be stored. The data_folder option was created to facilitate data storage on external drives. |

In [87]:
project_folder = '/Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples'
data_folder='/Volumes/Seagate/CHANGES/data_repository/tutorial'

Initialize the IcesheetCHANGES object - this object will contain all pertinent information to initialize the data grids in your region of interest.

In [88]:
class IcesheetCHANGES:
    def __init__(self, project_folder, data_folder, region_name):

        #this initiates the global domain and all of its associated parameters
        self.project_folder = project_folder
        self.data_folder = data_folder
        self.region_name = region_name
        self.velocity_grid_x = []
        self.velocity_grid_y = []
        self.elevation_grid_y = []
        self.elevation_grid_x = []

In [89]:
IC = IcesheetCHANGES(project_folder,data_folder, region_name)

## Obtain list of available files for download

In [90]:
# this function is from the ICESat2 data page
def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    
    entries = [e['links']
                for e in search_results['feed']['entry']
                if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue    # continue jumps to next iteration in the loop
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue
        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])
    return urls

In [91]:
# Build and call the CMR API URL
cmr_query_url = 'https://cmr.earthdata.nasa.gov/search/granules.json?echo_collection_id=' + collection_id + '&page_size=2000'
response = requests.get(cmr_query_url)

# print error code based on response
if response.status_code != 200:
    print('ERROR: {}'.format(response.status_code))
search_page = response.json()
# If JSON contains an error message, print the message at the key, 'error'
if 'errors' in search_page:
    print(search_page['errors'])
else: 
    urls = cmr_filter_urls(search_page)
    print("Successfully obtained {} URLs.".format(len(urls)))

Successfully obtained 32 URLs.


In [92]:
# Get only .nc files and also store the file names in a seperate list
measures_antarctic_IVM_file_links = []
measures_antarctic_IVM_file_names = []
metadata = []
for url in urls:
    if '.nc'in url and not 'xml' in url:
        measures_antarctic_IVM_file_links.append(url)
        url_parts = url.split('/')
        file_name = url_parts[-1]
        measures_antarctic_IVM_file_names.append(file_name)

In [93]:
## write available file names and links to csv file
if not os.path.exists(os.path.join(IC.project_folder,IC.region_name,'Velocity','Metadata')):
    os.makedirs(os.path.join(IC.project_folder,IC.region_name,'Velocity','Metadata'))
f = open(os.path.join(IC.project_folder,IC.region_name,'Velocity','Metadata',
                                IC.region_name + '_MEaSUREs_Antarctic_annual.csv'),'w')

f.write('File_Name,URL')
for ea in range(len(measures_antarctic_IVM_file_names)):
    f.write('\n'+measures_antarctic_IVM_file_names[ea]+','+measures_antarctic_IVM_file_links[ea])
f.close()

In [94]:
# check for existing files, get a list of files not on disk
def obtain_download_list(file_names, file_links):
    
    output_folder=os.path.join(IC.data_folder,'Velocity','MEaSUREs','Annual','Data')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    download_list_names=[]
    download_list_links=[]
    for fil in file_names:
        download_file = True
        if fil not in os.listdir(os.path.join(output_folder)):
            for existing_fil in os.listdir(os.path.join(output_folder)):
                if fil == existing_fil:  # this allows for older versions to be kept 
                    download_file = False
        else:
            download_file=False
        if download_file:
            download_list_names.append(fil)
            download_list_links.append(file_links[file_names.index(fil)])
    #n_existing_files = len(file_names)-len(download_list)
    return(download_list_names, download_list_links) #,n_existing_files)

In [95]:
dl_list_names, dl_list_links = obtain_download_list(measures_antarctic_IVM_file_names, measures_antarctic_IVM_file_links)

## Download the data

In [96]:
def download_nc_direct(file_names, file_links):
    not_downloaded_names = []
    not_downloaded_links = []
    print('Downloading ' + str(len(file_names)) + ' file(s)...')
    for i in range(len(file_names)):
        # progress report
        print('Downloading ' + str(i + 1) + '/' + str(len(file_names)) + ': ' + file_names[i], end='\r')
        # get request (download file)
        r = requests.get(file_links[i], allow_redirects=True)
        if r.status_code != 200:    # 200 is the standard response for successful HTTP requests
            print('ERROR: ' + str(r.status_code) + '\n')
            not_downloaded_names.append(file_names[i])
            not_downloaded_links.append(file_links[i])
        # write content to file
        open(os.path.join(IC.data_folder,'Velocity','MEaSUREs','Annual','Data',file_names[i]), 'wb').write(r.content)
    return(not_downloaded_names, not_downloaded_links)

In [97]:
def run_download_nc_direct(dl_list_names, dl_list_links):
    # Download the files
    not_downloaded_names, not_downloaded_links = download_nc_direct(dl_list_names, dl_list_links)

    # Attempt to download any files that were not downloaded the first time
    if len(not_downloaded_names) > 0:
        print('The following files were not downloaded:')
        for i in range(len(not_downloaded_names)):
            print(not_downloaded_names[i] + ': ' + not_downloaded_links[i])
            
        print('Attempting to download these files once more...')
        not_downloaded_names, not_downloaded_links = download_nc_direct(not_downloaded_names, not_downloaded_links)
        if len(not_downloaded_names) > 0:
            print('Please download these files manually and place them in the following folder:')
            print(os.path.join(IC.data_folder,'Velocity','MEaSUREs','Annual','Data'))


In [98]:
run_download_nc_direct(dl_list_names, dl_list_links)